## Setup

In [ ]:
!pip install poetry

In [ ]:
!pip install https://github.com/swanandlab/SMorph/releases/download/v0.1.1-alpha/SMorph-0.1.1.tar.gz

In [ ]:
on_colab = 'google.colab' in str(get_ipython())

import warnings
warnings.filterwarnings('ignore')

if not on_colab:
    import napari
import smorph.util.autocrop as ac

In [ ]:
def view_3D(*args):  # Models a 3D image
    n_images = len(args)
    if not on_colab and n_images > 0:
        with napari.gui_qt():
            viewer = napari.view_image(**args[0], ndisplay=3)
            for itr in range(1, n_images):
                del args[itr]['colormap']
                del args[itr]['gamma']
                viewer.add_labels(**args[itr])

## 1. Import Confocal Microscopic Image of the Tissue

Set `CONFOCAL_TISSUE_IMAGE` to the path of the image file to be processed.
- Followed by non-local means denoising using auto-calibrated parameters

In [ ]:
CONFOCAL_TISSUE_IMAGE = 'Datasets/Garima Confocal/SAL,DMI, FLX ADN HALO_TREATMENT_21 DAYS/allImg/CA3/MSP2.1MA_1_SINGLE MARK_20X_SEC 1_RIGHT_CA3.czi'

original = ac.import_confocal_image(CONFOCAL_TISSUE_IMAGE)

denoiser = ac.calibrate_nlm_denoiser(original)
denoise_parameters = denoiser.keywords['denoiser_kwargs']
print(denoise_parameters)
denoised = ac.denoise(original, denoise_parameters)
ac.projectXYZ(denoised, .5, .5, 1)

In [ ]:
view_3D({'data': original, 'colormap': 'gray', 'name': 'original'},
        {'data': denoised, 'colormap': 'inferno', 'name': 'denoised'})

## 2. Select ROI using Polygonal Lasso Tool

Set two variables:
- `SELECT_ROI`: True, If you want to select ROI manually; else False
- `NAME_ROI`: Name of the manually selected ROI

In [ ]:
SELECT_ROI = True
NAME_ROI = 'CA3'
FILE_ROI = 'Datasets/Garima Confocal/SAL,DMI, FLX ADN HALO_TREATMENT_21 DAYS/allRoi/CA3/MSP2.1MA_1_SINGLE MARK_20X_SEC 1_RIGHT_CA3.roi'

NAME_ROI = NAME_ROI if SELECT_ROI else ''
IMG_NAME = CONFOCAL_TISSUE_IMAGE.split('/')[-1].split('.')[0]
linebuilder = None if not SELECT_ROI else ac.select_ROI(denoised, IMG_NAME + '-' + NAME_ROI, FILE_ROI)

In [ ]:
%matplotlib inline
if SELECT_ROI:
    original, denoised = ac.mask_ROI(original, denoised, linebuilder)
    ac.projectXYZ(denoised, .5, .5, 1)

## 3. Segmentation

### 3.1 Threshold & color label cells

Set two parameters:
- `LOW_THRESH`: Pixel intensity value corresponding to faintest branch's edge
- `HIGH_THRESH`: Pixel intensity value corresponding to faintest soma

Understand their effect by configuring three parameters:
- `LOW_DELTA`: Pixel intensity value corresponding to change in `LOW_THRESH`
- `HIGH_DELTA`: Pixel intensity value corresponding to change in `HIGH_THRESH`
- `N_STEPS`: Number of steps of delta in threshold to take in both directions

In [ ]:
LOW_THRESH = .05
HIGH_THRESH = .2

LOW_DELTA = .005
HIGH_DELTA = .00
N_STEPS = 2

results = ac.testThresholds(denoised, .5, .5, 1, 'gist_earth',
                            LOW_THRESH, HIGH_THRESH, LOW_DELTA, HIGH_DELTA, N_STEPS)
# view_3D({'data': denoised, 'colormap': 'inferno', 'name': 'denoised'}, *results)

Thresholding results

In [ ]:
thresholded = ac.threshold(denoised, LOW_THRESH, HIGH_THRESH)
labels = ac.label_thresholded(thresholded)
filtered_labels = ac.filter_labels(labels, thresholded, linebuilder, True)

In [ ]:
prefiltering_volume = thresholded.sum()
postfiltering_volume = (filtered_labels > 0).sum()

f'Prefiltering Volume: {prefiltering_volume}; ' \
f'Postfiltering Volume: {postfiltering_volume}; ' \
f'%age loss in volume: {(prefiltering_volume - postfiltering_volume) / prefiltering_volume * 100} %'

### 3.2 Filter segmented individual cells by removing ones in borders (touching the convex hull)

In [ ]:
# get the centroids and label values from the label image
regions = ac.arrange_regions(filtered_labels)
residue_regions = ac.arrange_regions(labels - filtered_labels)
centroid_coords = [r.centroid for r in regions]

# store the labels for each blob in a properties dictionary
pts_properties = {'obj': [i for i in range(len(regions))]}

### 3.3 Visualize segmented cells to determine cutoff volumes

#### 3.3.1 Check segmented cells on whole image

In [ ]:
if not on_colab:
    with napari.gui_qt():
        viewer = napari.view_image(denoised, name='denoised')
        viewer.add_labels(filtered_labels, name='filtered_labels')
        viewer.add_points(centroid_coords, edge_color='transparent',
                          face_color='transparent',
                          properties=pts_properties, text='obj')

#### 3.3.2 Check batches of objects

In [ ]:
N_BATCHES = ac.paginate_objs(regions, pg_size=50)

In [ ]:
# Set `BATCH_NO` to view detected objects in paginated 2D MIP views.
BATCH_NO = 1
ac.project_batch(BATCH_NO, N_BATCHES, regions, denoised)

#### 3.3.2 Check individual objects
Select individual objects using `OBJ_INDEX`.

In [ ]:
OBJ_INDEX = 189

extracted_cell = ac.extract_obj(regions[OBJ_INDEX], denoised)
minz, miny, minx, maxz, maxy, maxx = regions[OBJ_INDEX].bbox
ac.projectXYZ(extracted_cell, .5, .5, 1)
minz, miny, minx, maxz, maxy, maxx

In [ ]:
view_3D({'data': denoised[minz:maxz, miny:maxy, minx:maxx], 'name': 'denoised'},
        {'data': regions[OBJ_INDEX].image, 'colormap': 'yellow', 'name': 'thresholded'},
        {'data': extracted_cell, 'colormap': 'inferno', 'name': 'extracted_cell'})

## 4. Export autocropped 3D cells or 2D max intensity projections

Set two parameters:
- `LOW_VOLUME_CUTOFF`: to filter out noise/artifacts
- `HIGH_VOLUME_CUTOFF`: to filter out cell clusters

For choosing between 3D segmented cells or 2D max intensity projections:
- Set `OUTPUT_OPTION` = '3d' for 3D cells, or
- Set `OUTPUT_OPTION` = 'mip' for Max Intensity Projections.

In [ ]:
LOW_VOLUME_CUTOFF = 143  # filter noise/artifacts
HIGH_VOLUME_CUTOFF = 1245  # filter cell clusters

OUTPUT_OPTION = 'both'  # '3d' for 3D cells, 'mip' for Max Intensity Projections
SEGMENT_TYPE = 'segmented'

ac.export_cells(CONFOCAL_TISSUE_IMAGE, LOW_VOLUME_CUTOFF,
                HIGH_VOLUME_CUTOFF, OUTPUT_OPTION, denoised,
                regions, residue_regions, SEGMENT_TYPE, NAME_ROI, linebuilder, FILE_ROI)

## Select somas of individual cells in clumps

In [ ]:
import numpy as np
SOMA_SELECTED = 'Autocropped/MSP2.1MA_1_SINGLE MARK_20X_SEC 1_RIGHT_CA3-CA3/residue'
reconstructed_labels = np.zeros(original.shape)
reconstructed_labels, parent_path, roi_path = ac.postprocess_segment(SOMA_SELECTED, reconstructed_labels)

In [ ]:
view_3D({'data': labels, 'colormap': 'gray', 'name': 'labels'},
        {'data': reconstructed_labels, 'colormap': 'inferno', 'name': 'reconstructed_labels'})

In [ ]:
reconstructed_volume = (reconstructed_labels > 0).sum()
f'Postreconstructing Volume: {reconstructed_volume}; ' \
f'%age volume reconstructed: {(reconstructed_volume) / prefiltering_volume * 100} %'

In [ ]:
linebuilder = ac._roi_extract._load_ROI(roi_path)

In [ ]:
reconstructed_residue_labels = ac.filter_labels(reconstructed_labels, thresholded, linebuilder, True)

In [ ]:
reconstructed_residue_volume = (reconstructed_residue_labels > 0).sum()
f'Postreconstructing-residue Volume: {reconstructed_residue_volume}; ' \
f'%age volume: {(reconstructed_residue_volume) / prefiltering_volume * 100} %'

In [ ]:
view_3D({'data': denoised, 'colormap': 'inferno', 'name': 'denoised'},
        {'data': reconstructed_residue_labels, 'colormap': 'gray', 'gamma': .8, 'name': 'reconstructed_filtered_labels'},
        {'data': filtered_labels, 'colormap': 'inferno', 'gamma': .8, 'name': 'filtered_labels'},
        {'data': labels, 'colormap': 'gist_earth', 'gamma': .8, 'name': 'labels'})

In [ ]:
reconstructed_filtered_regions = ac.arrange_regions(reconstructed_residue_labels)

LOW_VOLUME_CUTOFF = 0  # filter noise/artifacts
HIGH_VOLUME_CUTOFF = 1e9  # filter cell clusters

OUTPUT_OPTION = 'both'  # '3d' for 3D cells, 'mip' for Max Intensity Projections
SEGMENT_TYPE = 'segmented'

ac.export_cells(parent_path, LOW_VOLUME_CUTOFF,
                HIGH_VOLUME_CUTOFF, OUTPUT_OPTION, denoised,
                reconstructed_filtered_regions, None, SEGMENT_TYPE, NAME_ROI, linebuilder)